Finding the optimum sequence of least difficult courses using Ant colony optimisation, given the difficulty of proceeding from one course to another.

Here a course is analogous to destinations or nodes that an ant visits.
A student is analogous to an ant.

In [96]:
import random

In [166]:
class AntColony:
    def __init__(self, num_courses, course_credits, course_times, num_students, max_hours, min_credits_reqd, 
                 num_iterations, q, rho, alpha, beta, vis, pher):
        self.num_courses = num_courses
        self.course_credits = course_credits
        self.course_times = course_times
        self.num_students = num_students
        self.max_hours = max_hours
        self.min_credits_reqd = min_credits_reqd
        self.num_iterations = num_iterations
        self.q = q
        self.rho = rho
        self.alpha = alpha
        self.beta = beta
        self.vis = vis
        self.pher = pher
        
    def __next_course(self, courses_done_k, curr):
        course = -1
        r = random.random()
        cum_sum = 0   
        tau = []
        allowed_courses = 0
        for j in range(self.num_courses):
            if not(j in courses_done_k):
                tau_jk = (self.pher[curr][j]**self.alpha) * (self.vis[curr][j]**self.beta)
                tau.append(tau_jk)
        sigma = sum(tau)
        first = True
        for j in range(self.num_courses):
            if not(j in courses_done_k):
                if (first):
                    course = j
                    first = False
                prob = tau[allowed_courses] / sigma
                allowed_courses += 1
                cum_sum += prob
                if (r >= cum_sum):
                    course = j
        return course
    
    def __update(self, dist, route):
        for i in range(self.num_courses):
            for j in range(self.num_courses):
                delta = 0
                for k in range(self.num_students):
                    if (i,j) in route[k]:
                        delta += dist[k]
                self.pher[i][j] = (1-self.rho)*self.pher[i][j] + delta
                
    def run(self):
        for t in range(self.num_iterations):
            dist = []
            courses_done = []
            route = []
            for k in range(self.num_students):
                course_k = []
                route_k = []
                courses_done.append(course_k)
                route.append(route_k)
            
            for k in range(self.num_students):
                dist_k = 0
                curr = 0
                # Starting with course 0
                courses_done[k].append(0)
                credit = self.course_credits[0]
                time = self.course_times[0]
                for j in range(self.num_courses-1):
                    c = self.__next_course(courses_done[k], curr)
                    credit += self.course_credits[c]
                    time += self.course_times[c]
                    route[k].append((curr, c))
                    courses_done[k].append(c)
                    dist_k += cost[curr][c]
                    if (time > self.max_hours):
                        print("Not possible with given constraints")
                        break
                    if (credit >= self.min_credits_reqd):
                        break
                    curr = c
                if (credit >= self.min_credits_reqd and time <= self.max_hours):
                    dist.append(self.q * credit/dist_k)
                else:
                    dist.append(0)
            print(f"Route after iteration {t}:")
            print(route[self.num_students-1])
            print(dist)
            self.__update(dist, route)
            print("Updated pheromone levels:")
            for ph in self.pher:
                print(ph)
            print()

In [170]:
def main():
    # Taking Input
    num_courses = int(input("Enter the number of courses: "))
    print("Enter the credits for each course: ")
    course_credits = []
    for i in range(num_courses):
        course_credits.append(int(input()))
    course_times = []
    print("Enter the hours required for each course: ")
    for i in range(num_courses):
        course_times.append(int(input()))
    num_students = int(input("Enter the number of students: "))
    max_hours = int(input("What is the maximum time you can dedicate in hours? "))
    min_credits_reqd = int(input("What is the minimum no of credits you require? "))
    num_iterations = int(input("How many iterations would you like to run? "))
    q = int(input("Enter Q value: "))
    rho = float(input("Enter pheromone evaporation rate: "))
    alpha = float(input("Enter alpha (pheromone influence): "))
    beta = float(input("Enter beta (difficulty and credits influence): "))
    
    diff = [] 
    print("Enter the difficulty matrix rowwise") 
    for i in range(num_courses):           
        aux =[] 
        for j in range(num_courses):      
            aux.append(int(input())) 
        diff.append(aux) 
    
    # Calculating visbility values
    vis = []
    for i in range(num_courses):
        aux = []
        for j in range(num_courses):
            if (diff[i][j] == 0):
                aux.append(0)
            else:
                aux.append(course_credits[j]/diff[i][j])
        vis.append(aux)
        
    # Initial Pheromone matrix
    pher = [[1,1,1,1,1],
            [1,1,1,1,1],
            [1,1,1,1,1],
            [1,1,1,1,1],
            [1,1,1,1,1]]   
    
    ant = AntColony(num_courses, course_credits, course_times, num_students, max_hours, min_credits_reqd,
                  num_iterations, q, rho, alpha, beta, vis, pher)
    # ant = AntColony(5, credits, times, 3, 200, 13, 3, 1, 0.2, 1, 2, h, c_ph)
    ant.run()

In [171]:
main()

Enter the number of courses: 5
Enter the credits for each course: 
4
6
5
1
7
Enter the hours required for each course: 
10
34
25
61
42
Enter the number of students: 3
What is the maximum time you can dedicate in hours? 200
What is the minimum no of credits you require? 13
How many iterations would you like to run? 3
Enter Q value: 1
Enter pheromone evaporation rate: 0.2
Enter alpha (pheromone influence): 1
Enter beta (difficulty and credits influence): 2
Enter the difficulty matrix rowwise
0
10
12
11
14
10
0
13
15
8
12
13
0
9
14
11
15
9
0
16
14
8
14
16
0
Route after iteration 0:
[(0, 1), (1, 3), (3, 2)]
[0.47058823529411764, 0.47058823529411764, 0.47058823529411764]
Updated pheromone levels:
[0.8, 2.211764705882353, 0.8, 0.8, 0.8]
[0.8, 0.8, 0.8, 2.211764705882353, 0.8]
[0.8, 0.8, 0.8, 0.8, 0.8]
[0.8, 0.8, 2.211764705882353, 0.8, 0.8]
[0.8, 0.8, 0.8, 0.8, 0.8]

Route after iteration 1:
[(0, 1), (1, 3), (3, 2)]
[0.47058823529411764, 0.47058823529411764, 0.47058823529411764]
Updated pher